In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer
import torch
# tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-base")
# model = T5ForConditionalGeneration.from_pretrained("google/t5-v1_1-base")
# model.parallelize()
model_cache_dir = '/work/09127/tomyoung/ls6/inconsistencies_project/t5-base-cache'
model = T5ForConditionalGeneration.from_pretrained("google/ul2", cache_dir=model_cache_dir).to("cuda")
model.parallelize()                                                                                                  



In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google/ul2")

# t5 tokenizer
t5_tokenizer = T5Tokenizer.from_pretrained("t5-base")


In [ ]:
len(tokenizer.get_vocab())

In [ ]:
len(t5_tokenizer.get_vocab())

In [ ]:
# inputs = tokenizer("[NLG] Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, solid man wiht a bald head. Mrs. Dursley was thin and blonde and more than the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbours. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere. <extra_id_0>", return_tensors="pt").input_ids.to("cuda")
inputs = tokenizer("Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, solid man wiht a bald head. Mrs. Dursley was thin and blonde and more than the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbours. The Dursleys had a small son called Dudley and in their opinion there was no finer boy anywhere. <extra_id_0>", return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(inputs, output_scores=True, return_dict_in_generate=True)
# input_string = ""                                           
outputs_beam = model.generate(inputs, num_beams=2, num_return_sequences=2, output_scores=True, return_dict_in_generate=True)
labels = tokenizer("<extra_id_0> Dudley was a very good-looking child, with a bald head", return_tensors="pt").input_ids.to("cuda")
outputs_forward = model(input_ids=inputs, labels=labels)
# outputs = model.generate(inputs, output_scores=True, return_dict_in_generate=True)
# outputs = model.generate(inputs, num_beams=2, max_length=3, num_return_sequences=2, output_scores=True, return_dict_in_generate=True)
loss_forward = outputs_forward.loss
logits_forward = outputs_forward.logits

In [ ]:
inputs = tokenizer("[NLG] I will be having a <extra_id_0>", return_tensors="pt").input_ids.to("cuda")
outputs = model.generate(inputs, output_scores=True, return_dict_in_generate=True, max_length=8)
outputs_beam = model.generate(inputs, num_beams=2, max_length=8, num_return_sequences=2, output_scores=True, return_dict_in_generate=True)
labels = tokenizer("<extra_id_0> sabbatical for", return_tensors="pt").input_ids.to("cuda")
labels = labels[:, :-1]
outputs_forward = model(input_ids=inputs, labels=labels)
loss_forward = outputs_forward.loss
logits_forward = outputs_forward.logits

In [ ]:
labels.shape

In [ ]:
import torch
import torch.nn.functional as F

def cross_entropy_loss(logits, labels):
    batch_size = logits.size(0)
    log_probs = F.log_softmax(logits, dim=1)

    loss = 0
    for i in range(batch_size):
        loss -= log_probs[i, labels[i]]
        #.item()
    loss /= batch_size
    return loss

In [ ]:
logits_forward[0].size(0)

In [ ]:
loss = cross_entropy_loss(logits_forward[0], labels[0])
print(loss)
print(loss_forward)

In [ ]:
log_probs = F.log_softmax(logits_forward[0], dim=1)
log_probs.shape

In [ ]:
batch_size = logits_forward[0].size(0)
log_probs = F.log_softmax(logits_forward[0], dim=1)


In [ ]:
-1.3116 * 8 / 7

In [ ]:
outputs_beam['sequences_scores']

In [ ]:
outputs_beam['scores']
# get the probs

In [ ]:
# verify that that the first one in each index in outputs_beam['scores'] is the same as in outputs['scores']
from termcolor import cprint

for i in range(len(outputs_beam['scores'])):
    # print(outputs_beam['scores'][i][0] == outputs['scores'][i])
    probs_outputs_beam_scores = torch.nn.functional.softmax(outputs_beam['scores'][i], dim=-1)
    print(probs_outputs_beam_scores[0])
    # argmax 
    print(torch.argmax(probs_outputs_beam_scores[0]))
    # decode it
    print(tokenizer.decode(torch.argmax(probs_outputs_beam_scores[0])))
    # the value
    print(probs_outputs_beam_scores[0][torch.argmax(probs_outputs_beam_scores[0])])
    probs_outputs_scores = torch.nn.functional.softmax(outputs['scores'][i], dim=-1)
    print(probs_outputs_scores[0])
    # argmax
    print(torch.argmax(probs_outputs_scores[0]))\
    # decode it
    print(tokenizer.decode(torch.argmax(probs_outputs_scores[0])))
    # the value
    print(probs_outputs_scores[0][torch.argmax(probs_outputs_scores[0])])
    print('-------------')
    # the relative difference
    print('relative difference:')
    print((probs_outputs_beam_scores[0][torch.argmax(probs_outputs_beam_scores[0])] - probs_outputs_scores[0][torch.argmax(probs_outputs_scores[0])]) / probs_outputs_scores[0][torch.argmax(probs_outputs_scores[0])])



In [ ]:
value = probs_outputs_scores[0][0].item()
# check its memory size
import sys
sys.getsizeof(value)
# check its binary representation
import struct
struct.pack('f', value)

In [ ]:
outputs['scores']
# tokenizer.batch_decode(labels)

In [ ]:
sequence_index = 1
outputs_beam["sequences"][sequence_index]
print(len(outputs_beam["sequences"][sequence_index]))
tokenizer.batch_decode(outputs_beam["sequences"][sequence_index])
tokenizer.decode(outputs_beam["sequences"][sequence_index])

In [ ]:
outputs["sequences"][1]
print(len(outputs["sequences"][1]))
tokenizer.batch_decode(outputs["sequences"][1])
tokenizer.decode(outputs["sequences"][1])

In [ ]:
outputs["sequences"][0]
# tokenizer.batch_decode(outputs["sequences"][0])
# tokenizer.decode(outputs["sequences"][0])

In [ ]:
outputs_beam['sequences_scores']

In [ ]:
outputs_beam['beam_indices']

In [ ]:
outputs_beam['scores']

In [ ]:
outputs_beam['scores']

In [ ]:
outputs['sequences']
tokenizer.batch_decode(outputs_beam['sequences'][0])
tokenizer.decode(outputs['sequences'][0])

In [ ]:
print(len(outputs['scores']))
for k in range(len(outputs['scores'])):
    outputs['scores'][k]
    probs = torch.softmax(outputs['scores'][k], dim=-1)
    best_id = torch.argmax(probs)
    print(best_id)
    print(tokenizer.decode(best_id))
    print(probs[0][best_id])
    # print(probs_forward[0][k][best_id])
    # print relative difference
    print('-----------')

In [ ]:
# print the beam scores for the first sequence
for k in range(len(outputs['scores'])):
    


In [ ]:
logits_forward
probs_forward = torch.softmax(logits_forward, dim=-1)
probs
# probs[0].shape

for k in range(logits_forward.shape[1]):
    print(labels[0][k])
    print(tokenizer.decode(labels[0][k]))
    print(probs_forward[0][k][labels[0][k]])
    print('------------')


In [ ]:
import math
def evaluate_conditional_probablity(model, tokenizer, input_string, target_string):
    input_ids = tokenizer(input_string, return_tensors="pt").input_ids
    labels = tokenizer(target_string, return_tensors="pt").input_ids
    outputs = model(input_ids=input_ids, labels=labels)
    loss = outputs.loss
    logits = outputs.logits
    likelihood = math.exp(-loss)
    return likelihood, loss, logits

In [ ]:
import math
# training
input_ids = tokenizer("A man is having a <extra_id_0> for breakfast.", return_tensors="pt").input_ids
labels = tokenizer("<extra_id_0> small pizza <extra_id_1>", return_tensors="pt").input_ids
outputs = model(input_ids=input_ids, labels=labels)
loss = outputs.loss
logits = outputs.logits

# recover likelihood from loss
likelihood = math.exp(-loss)
likelihood


In [ ]:
from transformers import T5Tokenizer, T5Model

tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5Model.from_pretrained("t5-base")

input_ids = tokenizer(
    "Studies have been shown that owning a dog is good for you", return_tensors="pt"
).input_ids  # Batch size 1
decoder_input_ids = tokenizer("Studies show that", return_tensors="pt").input_ids  # Batch size 1

# forward pass
outputs = model(input_ids=input_ids, decoder_input_ids=decoder_input_ids)
last_hidden_states = outputs.last_hidden_state


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
import transformers
transformers.__version__

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")



In [ ]:
input_ids = tokenizer("100 + 100 equals <extra_id_0>. ", return_tensors="pt").input_ids
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
# change the default huggingface caching directory


tokenizer = T5Tokenizer.from_pretrained("t5-large")
model = T5ForConditionalGeneration.from_pretrained("t5-large", cache_dir='./t5-large-cache')

input_ids = tokenizer("The <extra_id_0> walks in the park", return_tensors="pt").input_ids

sequence_ids = model.generate(input_ids)
sequences = tokenizer.batch_decode(sequence_ids)
sequences

In [ ]:
import pickle
acceptable_alternatives_bigrams = '/work/09127/tomyoung/ls6/data/pkls/acceptable_alternatives_1000_ignore_cws_nos_50.pkl'

with open(acceptable_alternatives_bigrams, 'rb') as f:
    acceptable_alternatives_bigrams = pickle.load(f)
acceptable_alternatives_bigrams.keys()

In [ ]:
len(acceptable_alternatives_bigrams)

In [ ]:
acceptable_alternatives_bigrams[(25, 3, 21)][0]

In [ ]:
acceptable_alternatives_bigrams[(25, 3, 22)]